#### Fonction pour Diviser Vos Données en Trois : CSV, JSON et Base de Données :

Cette fonction divise un CSV en trois parties : 
- une pourcentage pour JSON,
- une pourcentage pour la base de données,
- et le reste pour le CSV.

Requirement : 
- SQL SERVER .
- database alrady create .
- Dataset with format csv .
- Data already cleaning .


In [2]:
import pandas as pd
import pyodbc 
import json

In [8]:
# p_json              : Pourcentage de la partie JSON.
# p_Db                : Pourcentage de la partie base de données.
# csv_file_name       : Nom de votre fichier d'origine au format CSV.
# file_json_name_part : Nom du fichier JSON.
# server              : Nom de votre serveur SQL.
# database            : Nom de votre base de données.
# table_name          : Nom de votre table.
# file_Csv_name_part  : Nom du fichier CSV résultant."

def data_spliter(p_json, p_DB, csv_file_name, file_json_name_part, server, database, table_name, file_Csv_name_part):
    # Vérification des paramètres :
    if p_json + p_DB <= 100 and '.csv' in csv_file_name:
        # Lecture du fichier CSV :
        df = pd.read_csv(csv_file_name)

        # Suppression des valeurs NULL :
        df.dropna(axis=1, inplace=True)

        # Mélange aléatoire des lignes du DataFrame :
        df_random_order = df.sample(frac=1, random_state=42)

        # Liste des noms de toutes les colonnes :
        columns = list(df_random_order.columns)

        # Calcul du nombre de lignes pour chaque type (csv, json, db) :
        p_json = int(df_random_order.shape[0] * (p_json * 0.01))
        p_DB = int(df_random_order.shape[0] * (p_DB * 0.01))
        p_csv = df_random_order.shape[0] - (p_json + p_DB)

        # Création du fichier JSON :
        json_data_list = df_random_order[0:p_json].to_dict(orient='records')
        with open(f'{file_json_name_part}_{p_json}_row.json', 'w') as json_file:
            json.dump(json_data_list, json_file, indent=4)

        # Création de la Base de Données :
        db_data = df_random_order[p_json + 1:p_json + 1 + p_DB]
        driver = 'SQL Server'
        conn = pyodbc.connect(f'''
                                DRIVER={driver};
                                SERVER={server};
                                DATABASE={database};
                                Trusted_Connection=yes
                            ''')
        cursor = conn.cursor()

        # Création de la table :
        query = f"CREATE TABLE [{table_name}_{p_DB}_row] (\n"
        for col in columns:
            query += f"    {col} VARCHAR(MAX),\n"
        query = query[:-2] + "\n);"
        cursor.execute(query)
        cursor.commit()

        # Création et exécution des requêtes d'INSERT pour la base de données :
        insert_query = f"INSERT INTO [{table_name}_{p_DB}_row] ({', '.join(columns)}) VALUES ({', '.join(['?'] * len(columns))})"
        for _, row in db_data.iterrows():
            var = []
            for i in columns:
                var.append(row[i])
            cursor.execute(insert_query, var)
        cursor.commit()

        # Création du fichier CSV :
        if p_json + p_DB != 100:
            Csv_data = df_random_order[p_json + 1 + p_DB + 1:]
            df.to_csv(f'{file_Csv_name_part}_{Csv_data.shape[0]}_row.csv', index=False, header=False)

    else:
        print('Erreur de paramètres')

In [ ]:
data_spliter(58,15,'books.csv','mybook','LAPTOP-6D8J0VI4\SQLEXPRESS','book_db','book_table','my_csv_part')